In [4]:
import pandas as pd
import warnings
# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar

# ==================== CELL 1: ĐỌC DỮ LIỆU ====================
print("🔄 Bắt đầu đọc dữ liệu Parquet...")
parquet_folder = r"C:\Khue\TDN\data\processed" # <-- Đảm bảo đường dẫn này chính xác

# Tự động tìm tất cả các file parquet theo mẫu
try:
    all_files_in_folder = os.listdir(parquet_folder)
    parquet_files = [os.path.join(parquet_folder, f)
                     for f in all_files_in_folder
                     if f.startswith("sanluong_") and f.endswith(".parquet")]

    if not parquet_files:
        print(f"⚠️ Không tìm thấy file nào khớp mẫu 'sanluong_*.parquet' trong thư mục: {parquet_folder}")
        # Có thể dừng ở đây hoặc xử lý tiếp nếu cần
        # exit() # Bỏ comment nếu muốn dừng hẳn script
        df_all = pd.DataFrame() # Tạo DataFrame rỗng để code không lỗi ở các bước sau
    else:
        # Đọc và ghép toàn bộ file
        df_list = []
        for f in parquet_files:
            try:
                df_list.append(pd.read_parquet(f, engine='pyarrow')) # engine='pyarrow' thường là mặc định và nhanh
            except Exception as e:
                print(f"❌ Lỗi khi đọc file {f}: {e}")
        
        if df_list:
             df_all = pd.concat(df_list, ignore_index=True)
             print(f"✅ Đã đọc xong {len(parquet_files)} file parquet.")
             print(f"👉 Tổng số dòng dữ liệu ban đầu: {df_all.shape[0]:,}")
        else:
            print("❌ Không đọc được file nào thành công.")
            df_all = pd.DataFrame() # Tạo DataFrame rỗng

except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
    df_all = pd.DataFrame() # Tạo DataFrame rỗng
except Exception as e:
    print(f"❌ Lỗi không xác định khi truy cập thư mục hoặc file: {e}")
    df_all = pd.DataFrame() # Tạo DataFrame rỗng

# ==================== CELL 2: CHỌN CỘT VÀ ĐỔI TÊN ====================
if not df_all.empty:
    # Chọn các cột cần thiết và TẠO BẢN SAO (.copy()) để tránh SettingWithCopyWarning
    required_cols = ['CTDL', 'NMTD', 'MADIEMDO', 'ENDTIME', 'CS']
    if all(col in df_all.columns for col in required_cols):
        df_sanluong = df_all[required_cols].copy()
        # Đổi tên cột ENDTIME thành TIME
        df_sanluong.rename(columns={'ENDTIME': 'TIME'}, inplace=True)
    else:
        print(f"❌ Thiếu cột cần thiết trong dữ liệu. Cần có: {required_cols}")
        missing = [col for col in required_cols if col not in df_all.columns]
        print(f"   -> Các cột bị thiếu: {missing}")
        df_sanluong = pd.DataFrame() # Tạo rỗng để tránh lỗi sau
else:
    print("ℹ️ Bỏ qua bước chuẩn bị dữ liệu sản lượng do không có dữ liệu đầu vào.")
    df_sanluong = pd.DataFrame()

# ==================== CELL 3: XỬ LÝ THỜI GIAN, TẠO UI VÀ VẼ BIỂU ĐỒ ====================

# Chỉ tiếp tục nếu df_sanluong có dữ liệu
if not df_sanluong.empty:
    print("🔄 Xử lý thời gian và tối ưu DataFrame...")
    # ========== TỐI ƯU HÓA: XỬ LÝ DATETIME VÀ ĐẶT INDEX ==========
    # 1. Đảm bảo cột TIME là datetime
    try:
        df_sanluong['TIME'] = pd.to_datetime(df_sanluong['TIME'])
    except Exception as e:
        print(f"❌ Lỗi chuyển đổi cột TIME sang datetime: {e}")
        # Cân nhắc dừng hoặc xử lý lỗi khác tại đây
        df_sanluong = pd.DataFrame() # Đặt lại thành rỗng nếu không xử lý được time

    if not df_sanluong.empty:
        # 2. Tạo các cột thời gian để index
        df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
        df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month
        df_sanluong['DAY_NUM'] = df_sanluong['TIME'].dt.day

        # 3. Đặt MultiIndex (KHÔNG bao gồm 'TIME' vì cần nó làm cột cho trục X)
        #    Index sẽ giúp tăng tốc lọc bằng .loc
        index_cols = ['CTDL', 'NMTD', 'YEAR', 'MONTH_NUM', 'DAY_NUM']
        # Kiểm tra sự tồn tại của các cột trước khi đặt index
        if all(col in df_sanluong.columns for col in index_cols):
             # Giữ lại các cột cần thiết cho plot ('TIME', 'CS', 'MADIEMDO') làm cột thường
            df_sanluong.set_index(index_cols, inplace=True)
            
            # 4. Sắp xếp Index - BƯỚC QUAN TRỌNG để .loc nhanh
            df_sanluong.sort_index(inplace=True)
            print("✅ Đã xử lý thời gian và tối ưu index DataFrame.")
        else:
            print("❌ Thiếu cột cần thiết để tạo index tối ưu.")
            missing_idx_cols = [col for col in index_cols if col not in df_sanluong.columns]
            print(f"   -> Các cột index bị thiếu: {missing_idx_cols}")
            # Nếu muốn vẫn chạy nhưng không tối ưu, có thể bỏ qua set_index và sort_index
            # Nếu không, đặt df_sanluong thành rỗng
            df_sanluong = pd.DataFrame()


# --- Các phần còn lại chỉ chạy nếu df_sanluong sau khi xử lý vẫn có dữ liệu ---
if not df_sanluong.empty:
    print("🔄 Tạo giao diện tương tác...")
    # ========== DANH SÁCH CHO DROPDOWN ==========
    # Lấy từ index thay vì cột sau khi đã set_index
    try:
        ctdl_list = sorted(df_sanluong.index.get_level_values('CTDL').unique())
        year_list = sorted(df_sanluong.index.get_level_values('YEAR').unique())
    except Exception as e:
         print(f"❌ Lỗi khi lấy danh sách CTDL/Year từ index: {e}")
         ctdl_list = []
         year_list = []


    # ========== HÀM TẠO DROPDOWN ==========
    def make_dropdown(options, description, width='250px', margin='0px 20px 0px 0px'):
        # Đảm bảo options không phải là None hoặc kiểu không phù hợp
        safe_options = list(options) if options is not None else []
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            layout=widgets.Layout(width=width, margin=margin),
            style={'description_width': 'auto'},
            disabled=not bool(safe_options) # Vô hiệu hóa nếu không có lựa chọn
        )

    # ========== DROPDOWNS ==========
    ctdl_dropdown_month = make_dropdown(ctdl_list, 'CTDL:', '300px')
    nmtd_dropdown_month = make_dropdown([], 'Nhà máy:', '300px') # Bắt đầu rỗng
    year_dropdown_month = make_dropdown(year_list, 'Năm:', '120px')
    month_dropdown_month = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
    day_placeholder = widgets.Label(value='', layout=widgets.Layout(width='120px', margin='0px 20px 0px 0px')) # Để căn chỉnh

    ctdl_dropdown_day = make_dropdown(ctdl_list, 'CTDL:', '300px')
    nmtd_dropdown_day = make_dropdown([], 'Nhà máy:', '300px') # Bắt đầu rỗng
    year_dropdown_day = make_dropdown(year_list, 'Năm:', '120px')
    month_dropdown_day = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
    day_dropdown_day = make_dropdown([], 'Ngày:', '120px') # Bắt đầu rỗng, sẽ được cập nhật

    # ========== NHÓM WIDGET ==========
    month_widgets = [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]
    day_widgets = [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]

    # ========== CẬP NHẬT NHÀ MÁY (Tối ưu bằng .loc trên index) ==========
    def update_nmtd(ctdl_value, dropdown):
        options = [] # Mặc định là rỗng
        if ctdl_value and not df_sanluong.empty: # Chỉ cập nhật nếu có giá trị CTDL và DataFrame không rỗng
            try:
                # Lấy danh sách NMTD duy nhất tại level 'NMTD' của index, ứng với ctdl_value đã chọn
                # df_sanluong.loc[ctdl_value] trả về DataFrame con với index cấp 1 là ctdl_value
                options = sorted(df_sanluong.loc[ctdl_value].index.get_level_values('NMTD').unique())
            except KeyError:
                # Trường hợp ctdl_value không tồn tại trong index (dù không nên xảy ra nếu list đúng)
                options = []
            except Exception as e:
                # Bắt các lỗi khác có thể xảy ra
                print(f" Lỗi khi cập nhật NMTD cho CTDL '{ctdl_value}': {e}")
                options = []
        
        # Cập nhật options và trạng thái disabled
        dropdown.options = options
        dropdown.disabled = not bool(options)
        # Nếu options cập nhật thành rỗng, reset giá trị về None
        if not options:
            dropdown.value = None


    # Gắn sự kiện observe và gọi lần đầu để khởi tạo NMTD
    # Sử dụng lambda để truyền đúng dropdown cần cập nhật
    ctdl_dropdown_month.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_month), names='value')
    ctdl_dropdown_day.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_day), names='value')

    # Khởi tạo NMTD ban đầu (nếu ctdl_dropdown có giá trị)
    if ctdl_dropdown_month.value:
         update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
    if ctdl_dropdown_day.value:
         update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)


    # ========== GIỚI HẠN NGÀY ==========
    def update_day_options(*args): # Chấp nhận các đối số dư thừa từ observe
        year = year_dropdown_day.value
        month = month_dropdown_day.value
        current_day_value = day_dropdown_day.value # Lưu giá trị ngày hiện tại

        if year and month:
            try:
                max_day = calendar.monthrange(year, month)[1]
                day_options = list(range(1, max_day + 1))
                day_dropdown_day.options = day_options
                day_dropdown_day.disabled = False
                # Cố gắng giữ lại giá trị ngày đã chọn nếu nó còn hợp lệ
                if current_day_value is not None and current_day_value in day_options:
                    day_dropdown_day.value = current_day_value
                else:
                    # Nếu không hợp lệ hoặc chưa có, đặt thành ngày 1
                     day_dropdown_day.value = 1 if day_options else None
            except Exception as e:
                # Xử lý lỗi nếu năm/tháng không hợp lệ (dù khó xảy ra với dropdown số)
                print(f" Lỗi khi lấy số ngày cho {month}/{year}: {e}")
                day_dropdown_day.options = []
                day_dropdown_day.disabled = True
                day_dropdown_day.value = None
        else:
            # Nếu chưa chọn Năm hoặc Tháng, vô hiệu hóa chọn Ngày
            day_dropdown_day.options = []
            day_dropdown_day.disabled = True
            day_dropdown_day.value = None

    # Gắn sự kiện observe và gọi lần đầu để khởi tạo Ngày
    year_dropdown_day.observe(update_day_options, names='value')
    month_dropdown_day.observe(update_day_options, names='value')
    update_day_options() # Gọi để khởi tạo lần đầu

    # ========== VÙNG OUTPUT CHO BIỂU ĐỒ ==========
    # Đặt trước để hàm plot_filtered có thể truy cập
    out = widgets.Output()

    # ========== VẼ BIỂU ĐỒ (Tối ưu bằng .loc trên index) ==========
    def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
        # Xóa output cũ trước khi bắt đầu
        with out:
            clear_output(wait=True) # wait=True giảm nhấp nháy

            # Kiểm tra đầu vào cơ bản
            if not all([ctdl, nmtd, year, month]):
                print("Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
                return
            if mode == 'day' and not day:
                print("Vui lòng chọn Ngày để xem theo ngày.")
                return
            

            filtered_df = pd.DataFrame() # Khởi tạo DataFrame rỗng

            try:
                if mode == 'month':
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM)
                    # reset_index() để lấy lại các cột index và các cột dữ liệu thường
                    idx_query = (ctdl, nmtd, year, month)
                    if idx_query in df_sanluong.index: # Kiểm tra nhanh sự tồn tại của key chính
                         filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{month}/{year}"
                else: # mode == 'day'
                    # Lọc bằng .loc trên index (CTDL, NMTD, YEAR, MONTH_NUM, DAY_NUM)
                    idx_query = (ctdl, nmtd, year, month, day)
                    if idx_query in df_sanluong.index:
                        filtered_df = df_sanluong.loc[idx_query].reset_index()
                    title_ext = f"{day}/{month}/{year}"

            except KeyError:
                # Xử lý trường hợp tổ hợp index không tồn tại (dù đã kiểm tra sơ bộ)
                print(f"ℹ️ Không tìm thấy dữ liệu chính xác cho lựa chọn (KeyError).")
                filtered_df = pd.DataFrame() # Đảm bảo là DataFrame rỗng
            except Exception as e:
                print(f"❌ Lỗi khi lọc dữ liệu bằng .loc: {e}")
                filtered_df = pd.DataFrame() # Đảm bảo là DataFrame rỗng

            # Kiểm tra lại lần nữa sau khi lọc và reset_index
            if filtered_df.empty:
                print("ℹ️ Không có dữ liệu cho lựa chọn này.")
                return # Thoát hàm nếu không có dữ liệu

            # --- Vẽ biểu đồ (Giữ nguyên cấu hình) ---
            try:
                # Sắp xếp lại theo TIME sau khi reset_index (quan trọng cho biểu đồ đường)
                filtered_df = filtered_df.sort_values('TIME')

                fig = px.line(
                    filtered_df,
                    x='TIME',
                    y='CS',
                    color='MADIEMDO', # Giữ nguyên color
                    render_mode='webgl', # Giữ nguyên render_mode
                    markers=True if mode == 'day' else False # Giữ nguyên logic markers
                )

                fig.update_layout(
                    title=None, # Giữ nguyên
                    annotations=[
                        dict(
                            text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                            xref='paper', yref='paper',
                            x=0.45, y=1.1,
                            xanchor='center', yanchor='top',
                            showarrow=False,
                            font=dict(size=18, color='black')
                        )
                    ],
                    template='plotly_white', # Giữ nguyên
                    height=600, # Giữ nguyên
                    hovermode='x unified', # Giữ nguyên

                    xaxis_title=None, # Giữ nguyên
                    yaxis_title='Giá trị công suất', # Giữ nguyên

                    xaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        title_font=dict(size=14, color='black'), # Bỏ bold vì đặt lại bằng annotation
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),
                    yaxis=dict( # Giữ nguyên
                        showgrid=True, gridcolor='lightgrey', griddash='dot',
                        rangemode='tozero',
                        title_font=dict(size=14, color='black', weight='bold'),
                        ticklabelposition='outside', ticks='outside', ticklen=8
                    ),

                    margin=dict(l=60, r=40, t=80, b=80), # Giữ nguyên
                    paper_bgcolor='white', # Giữ nguyên
                    plot_bgcolor='#f0f8ff' # Giữ nguyên
                )

                # ✅ Tên trục X bằng annotation (Giữ nguyên)
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref='paper', yref='paper',
                    x=0.45, y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color='black')
                )

                fig.update_traces( # Giữ nguyên
                    line=dict(width=2),
                    hovertemplate='Thời gian: %{x}<br>Công suất: %{y}'
                )
                
                fig.show() # Hiển thị biểu đồ trong Output 'out'

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ Plotly: {e}")
                # Có thể hiển thị lỗi chi tiết hơn nếu cần debug

    # ========== HÀM XỬ LÝ SỰ KIỆN THAY ĐỔI DROPDOWN ==========
    def on_change_month(change):
        # Kiểm tra xem các widget cần thiết có giá trị không
        if all(w.value is not None for w in [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]):
             # Chỉ gọi plot nếu là sự thay đổi có ý nghĩa (không phải reset về None)
            if change is None or change['new'] != change['old']:
                 plot_filtered(
                    'month',
                    ctdl_dropdown_month.value,
                    nmtd_dropdown_month.value,
                    year_dropdown_month.value,
                    month_dropdown_month.value
                 )
        # else: # Debugging (optional)
             # print("Debug Month: Một hoặc nhiều widget tháng chưa có giá trị.")


    def on_change_day(change):
        if all(w.value is not None for w in [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]):
            if change is None or change['new'] != change['old']:
                plot_filtered(
                    'day',
                    ctdl_dropdown_day.value,
                    nmtd_dropdown_day.value,
                    year_dropdown_day.value,
                    month_dropdown_day.value,
                    day_dropdown_day.value
                )
        # else: # Debugging (optional)
             # print("Debug Day: Một hoặc nhiều widget ngày chưa có giá trị.")


    # ========== HÀM TẮT / BẬT OBSERVER ==========
    # Giữ nguyên các hàm này, chúng quan trọng cho logic chuyển tab
    _observed_callbacks_month = {}
    _observed_callbacks_day = {}

    def disable_observe(widgets, callback_map, callback_func):
        for w in widgets:
            try:
                # Lưu trữ hàm callback để có thể gắn lại đúng hàm
                callback_map[w] = callback_func
                w.unobserve(callback_func, names='value')
                # print(f"Unobserved {w.description}") # Debug
            except ValueError:
                # print(f"Callback already removed for {w.description}") # Debug
                pass # Bỏ qua nếu callback đã được gỡ bỏ trước đó

    def enable_observe(widgets, callback_map):
        for w in widgets:
            if w in callback_map:
                try:
                    # Chỉ observe nếu widget không bị disabled
                    if not w.disabled:
                        w.observe(callback_map[w], names='value')
                        # print(f"Observed {w.description}") # Debug
                    # else:
                        # print(f"Skipped observe on disabled {w.description}") # Debug

                except Exception as e:
                    print(f"Error observing {w.description}: {e}") # Debug

    # ========== GẮN OBSERVER BAN ĐẦU ==========
    # Thay vì lặp, gọi enable_observe để gắn kèm kiểm tra disabled
    # Tạo map trước khi gọi enable_observe
    for w in month_widgets: _observed_callbacks_month[w] = on_change_month
    for w in day_widgets: _observed_callbacks_day[w] = on_change_day

    enable_observe(month_widgets, _observed_callbacks_month)
    enable_observe(day_widgets, _observed_callbacks_day)

    # ========== TẠO GIAO DIỆN ==========
    tab_thang = widgets.HBox([
        ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month, day_placeholder
    ], layout=widgets.Layout(justify_content='flex-start', align_items='flex-end', overflow='visible')) # Căn chỉnh đáy widget

    tab_ngay = widgets.HBox([
        ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day
    ], layout=widgets.Layout(justify_content='flex-start', align_items='flex-end', overflow='visible')) # Căn chỉnh đáy widget


    tabs = widgets.Tab(children=[tab_thang, tab_ngay])
    tabs.set_title(0, '📅 Xem theo tháng')
    tabs.set_title(1, '🗓️ Xem theo ngày')

    # ========== XỬ LÝ CHUYỂN TAB ==========
    def on_tab_change(change):
        if change['name'] == 'selected_index':
            new_tab_index = change['new']
            print(f"\n🔄 Chuyển sang tab: {tabs.get_title(new_tab_index)}")

            if new_tab_index == 1: # Chuyển sang tab NGÀY
                print("   Đồng bộ hóa lựa chọn từ Tháng sang Ngày...")
                # 1. Tắt observer của tab Ngày để tránh kích hoạt khi cập nhật giá trị
                disable_observe(day_widgets, _observed_callbacks_day, on_change_day)

                # 2. Đồng bộ giá trị từ tab Tháng sang tab Ngày
                ctdl_dropdown_day.value = ctdl_dropdown_month.value
                # update_nmtd sẽ được trigger bởi observe của ctdl_day khi bật lại,
                # nhưng ta cần gọi update trực tiếp để lấy đúng NMTD list trước
                update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)
                # Chỉ gán NMTD nếu giá trị tháng tồn tại trong options mới
                if nmtd_dropdown_month.value in nmtd_dropdown_day.options:
                     nmtd_dropdown_day.value = nmtd_dropdown_month.value
                else:
                     nmtd_dropdown_day.value = nmtd_dropdown_day.options[0] if nmtd_dropdown_day.options else None # Chọn cái đầu tiên hoặc None

                year_dropdown_day.value = year_dropdown_month.value
                month_dropdown_day.value = month_dropdown_month.value
                # update_day_options sẽ tự động được gọi khi month/year thay đổi (nếu observe đang bật)
                # Gọi lại để chắc chắn ngày được cập nhật đúng
                update_day_options()
                # Giá trị ngày sẽ được đặt về 1 (hoặc giá trị hợp lệ đầu tiên) bởi update_day_options

                # 3. Bật lại observer cho tab Ngày
                enable_observe(day_widgets, _observed_callbacks_day)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Ngày với giá trị mới
                print("   Vẽ biểu đồ cho tab Ngày...")
                on_change_day(None) # None để chỉ ra đây là gọi thủ công

            elif new_tab_index == 0: # Chuyển sang tab THÁNG
                print("   Đồng bộ hóa lựa chọn từ Ngày sang Tháng...")
                # 1. Tắt observer của tab Tháng
                disable_observe(month_widgets, _observed_callbacks_month, on_change_month)

                # 2. Đồng bộ giá trị từ tab Ngày sang tab Tháng
                ctdl_dropdown_month.value = ctdl_dropdown_day.value
                update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
                if nmtd_dropdown_day.value in nmtd_dropdown_month.options:
                    nmtd_dropdown_month.value = nmtd_dropdown_day.value
                else:
                    nmtd_dropdown_month.value = nmtd_dropdown_month.options[0] if nmtd_dropdown_month.options else None

                year_dropdown_month.value = year_dropdown_day.value
                month_dropdown_month.value = month_dropdown_day.value

                # 3. Bật lại observer cho tab Tháng
                enable_observe(month_widgets, _observed_callbacks_month)

                # 4. Kích hoạt vẽ lại biểu đồ cho tab Tháng
                print("   Vẽ biểu đồ cho tab Tháng...")
                on_change_month(None)

    tabs.observe(on_tab_change, names='selected_index')

    # ========== HIỂN THỊ ==========
    print("✅ Giao diện sẵn sàng!")
    display(tabs, out)

    # ========== VẼ BIỂU ĐỒ BAN ĐẦU ==========
    # Gọi on_change_month để vẽ biểu đồ lần đầu tiên khi cell chạy xong (cho tab tháng mặc định)
    on_change_month(None)

elif df_all.empty:
     print("\n❌ Không có dữ liệu đầu vào từ file Parquet để xử lý và vẽ biểu đồ.")
else:
     print("\n❌ Đã xảy ra lỗi trong quá trình chuẩn bị dữ liệu. Không thể tạo giao diện.")

🔄 Bắt đầu đọc dữ liệu Parquet...
✅ Đã đọc xong 4 file parquet.
👉 Tổng số dòng dữ liệu ban đầu: 18,404,864
🔄 Xử lý thời gian và tối ưu DataFrame...
✅ Đã xử lý thời gian và tối ưu index DataFrame.
🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()